### Phi-2
#### RAG with LlamaIndex - Nvidia CUDA + WSL (Windows Subsystem for Linux) + Word documents + Local LLM

### Note: Uses FP16 model (not quantized) - possible issue with llamaCPP and the quantized versions. I was sometimes not getting a response (even to the same question)

This notebook demonstrates the use of LlamaIndex for Retrieval Augmented Generation using Windows' WSL and an Nvidia's CUDA.

Environment:
- Windows 11
- Anaconda environment
- Nvidia RTX 3090
- LLMs - Phi-2, Mixtral 8x7B, Mistral 7B, Llama 2 13B Chat, Orca 2, Yi 34B - Quantized versions

Your Data:
- Add Word documents to the "Data" folder for the RAG to use

Package versions:
- See the "conda_package_versions.txt" for the full list of versions in the conda environment (generated using "conda list").
- See/utilise the "requirements.txt" file (note that you need to have installed the CUDA Toolkit using the instructions below, the versions are very important).

Local LLMs:
- I downloaded the quantized versions of the LLMs from huggingface.co - thanks to TheBloke who provided these quantized GGUF models. You can use higher quantized versions or different LLMs - just be aware that LLMs may have different prompt templates so be sure to use the correct prompt template format (e.g. Llama 2 requires a specific format for best results - see the Llama code for a function that creates the prompt).
- See the main README for links to all the models that I downloaded

Important libraries to "pip install":
- llama-cpp-python
- transformers
- llama-index
- docx2txt
- sentence-transformers

Notes:
Getting the Nvidia CUDA libraries installed correctly for use within WSL was challenging, I followed the steps from these two links:

1. CUDA Toolkit version 12.3 (latest as of 2023-11-03)
- https://docs.nvidia.com/cuda/wsl-user-guide/index.html
2. Install instructions within WSL:
- https://developer.nvidia.com/cuda-downloads?target_os=Linux&target_arch=x86_64&Distribution=WSL-Ubuntu&target_version=2.0&target_type=deb_local

To tell if you are utilising your Nvidia graphics card, in your command prompt, while in the conda environment, type "nvidia-smi". You should see your graphics card and you're notebook is running you should see your utilisation increase.

#### 1. Prepare Llama Index for use

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

#### 2. Load the Word document(s)

Note: A fictitious story about Thundertooth a dinosaur who has travelled to the future. Thanks ChatGPT!

In [2]:
documents = SimpleDirectoryReader("./Data/").load_data()

#### 3. Instantiate the model

In [3]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    model_url=None, # We'll load locally.
    model_path='./Models/phi-2-ggml-model-f16.gguf', # Using FP16 (not quantized) model as the quantized versions aren't always providing a response.
    temperature=0.1,
    max_new_tokens=256,
    context_window=2048, # Phi-2 2K context window - this could be a limitation for RAG as it has to put the content into this context window
    generate_kwargs={},
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 30}, # This is small model and there's no indication of layers offloaded to the GPU
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

llama_model_loader: loaded meta data with 18 key-value pairs and 325 tensors from ./Models/phi-2-ggml-model-f16.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight f16      [  2560, 51200,     1,     1 ]
llama_model_loader: - tensor    1:             blk.0.attn_norm.bias f32      [  2560,     1,     1,     1 ]
llama_model_loader: - tensor    2:           blk.0.attn_norm.weight f32      [  2560,     1,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_qkv.bias f32      [  7680,     1,     1,     1 ]
llama_model_loader: - tensor    4:            blk.0.attn_qkv.weight f16      [  2560,  7680,     1,     1 ]
llama_model_loader: - tensor    5:           blk.0.attn_output.bias f32      [  2560,     1,     1,     1 ]
llama_model_loader: - tensor    6:         blk.0.attn_output.weight f16      [  2560,  2560,     1,     1 ]
llama_model_loader: - tensor    7:                blk.0.ffn_up.bias f32      [ 10240,     1,     1, 

#### 4. Checkpoint

Are you running on GPU? The above output should include near the top something like:
> ggml_init_cublas: found 1 CUDA devices:

And in the full text near the bottom should be:
> llm_load_tensors: using CUDA for GPU acceleration

#### 5. Embeddings

Convert your source document text into embeddings.

The embedding model is from huggingface, this one performs well.

> https://huggingface.co/thenlper/gte-large


In [4]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-large
Load pretrained SentenceTransformer: thenlper/gte-large
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Use pytorch device: cuda


#### 6. Prompt Template

Prompt template for Phi-2 is below. As there's only a prompt we will combine the system message and prompt into the prompt.

Instruct: {prompt}<br>
Output:

In [20]:
# Produces a prompt specific to the model
def modelspecific_prompt(promptmessage):
    return f"Instruct: {promptmessage}\nOutput:"

#### 7. Service Context

For chunking the document into tokens using the embedding model and our LLM

In [21]:
service_context = ServiceContext.from_defaults(
    chunk_size=256, # Number of tokens in each chunk
    llm=llm,
    embed_model=embed_model
)

#### 8. Index documents

In [22]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

#### 9. Query Engine

Create a query engine, specifying how many citations we want to get back from the searched text (in this case 3).

The DB_DOC_ID_KEY is used to get back the filename of the original document

In [23]:
from llama_index.query_engine import CitationQueryEngine
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    citation_chunk_size=256,
)

# For citations we get the document info
DB_DOC_ID_KEY = "db_document_id"

#### 10. Prompt and Response function

Pass in a question, get a response back.

IMPORTANT: The prompt is set here, adjust it to match what you want the LLM to act like and do.

In [31]:
def RunQuestion(questionText):
    prompt = "" #"You are a story teller who likes to elaborate. Answer questions in a positive, helpful and interesting way. If the answer is not in the following context return ONLY 'Sorry, I don't know the answer to that'.\n\nAnswer the following question: "
    prompt = prompt + questionText

    queryQuestion = modelspecific_prompt(prompt)

    response = query_engine.query(queryQuestion)

    return response

#### 11. Questions to test with

In [41]:
TestQuestions = [
    "Summarise this story for me",
    "Who was the main protagonist?",
    "Did they have any children? If so, what were their names?",
    "Did anything eventful happen?",
]

#### 12. Run Questions through model (this can take a while) and see citations

Runs each test question, saves it to a dictionary for output in the last step.

Note: Citations are the source documents used and the text the response is based on. This is important for RAG so you can reference these documents for the user, and to ensure it's utilising the right documents.

In [42]:
qa_pairs = []

for index, question in enumerate(TestQuestions, start=1):
    question = question.strip() # Clean up

    print(f"\n{index}/{len(TestQuestions)}: {question}")

    response = RunQuestion(question) # Query and get  response

    qa_pairs.append((question.strip(), str(response).strip())) # Add to our output array

    # Displays the citations
    for index, node in enumerate(response.source_nodes, start=1):
        print(f"{index}/{len(response.source_nodes)}: |{node.node.metadata['file_name']}| {node.node.get_text()}")

    # Uncomment the following line if you want to test just the first question
    # break 


1/1: Summarise this story for me


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit


1/3: |Thundertooth Part 3.docx| Source 1:
Thundertooth



One fateful day, as the citizens of the futuristic city went about their daily lives, a collective gasp echoed through the streets as a massive meteor hurtled towards Earth. Panic spread like wildfire as people looked to the sky in horror, realizing the impending catastrophe. The city's advanced technology detected the threat, and an emergency broadcast echoed through the streets, urging everyone to seek shelter.



Thundertooth, ever the protector of his newfound home, wasted no time. With a determined gleam in his eyes, he gathered his family and hurried to the city's command center, where Mayor Grace and the leading scientists were coordinating the evacuation efforts.



The mayor, recognizing Thundertooth's intelligence and resourcefulness, approached him. "Thundertooth, we need a plan to divert or neutralize the meteor. Our technology can only do so much, but with your unique abilities, perhaps we can find a solution."



T


llama_print_timings:        load time =   19498.87 ms
llama_print_timings:      sample time =      45.65 ms /   102 runs   (    0.45 ms per token,  2234.59 tokens per second)
llama_print_timings: prompt eval time =     442.24 ms /     9 tokens (   49.14 ms per token,    20.35 tokens per second)
llama_print_timings:        eval time =   23614.77 ms /   101 runs   (  233.81 ms per token,     4.28 tokens per second)
llama_print_timings:       total time =   24390.68 ms


#### 13. Output responses

In [43]:
for index, (question, answer) in enumerate(qa_pairs, start=1):
    print(f"{index}/{len(qa_pairs)} {question}\n\n{answer}\n\n--------\n")

1/1 Summarise this story for me

The Thundertooth family, consisting of Lumina, Echo, Sapphire, and Ignis, used their unique abilities to divert a massive meteor that was heading towards Earth. With Lumina's force field, Echo's amplified warnings, Sapphire's calming presence, and Ignis's controlled bursts of flames, they successfully changed the course of the meteor, saving the city from destruction. The citizens expressed their gratitude, and the Thundertooth family basked in the relief of having protected their beloved home.

--------

